<a href="https://colab.research.google.com/github/proap900/AI_LABEXAM_Assignment/blob/main/_8_puzzle_by_bfs_and_manhattan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import deque
from copy import deepcopy
from IPython.display import display, clear_output
import ipywidgets as widgets

# --- Puzzle Setup ---
GOAL_8 = [[1,2,3],[4,5,6],[7,8,0]]

def manhattan(state):
    dist = 0
    for i in range(3):
        for j in range(3):
            val = state[i][j]
            if val != 0:
                goal_x, goal_y = divmod(val - 1, 3)
                dist += abs(goal_x - i) + abs(goal_y - j)
    return dist

def find_blank(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j

def move(state, direction):
    x, y = find_blank(state)
    dx, dy = {'up':(-1,0),'down':(1,0),'left':(0,-1),'right':(0,1)}[direction]
    nx, ny = x+dx, y+dy
    if 0 <= nx < 3 and 0 <= ny < 3:
        new = deepcopy(state)
        new[x][y], new[nx][ny] = new[nx][ny], new[x][y]
        return new
    return None

def serialize(state):
    return tuple(tuple(row) for row in state)

def draw_puzzle(state, move=None, h=None):
    output.clear_output(wait=True)
    with output:
        fig, ax = plt.subplots(figsize=(4, 4))
        ax.set_xlim(0, 3)
        ax.set_ylim(0, 3)
        ax.axis('off')
        for i in range(3):
            for j in range(3):
                val = state[i][j]
                color = 'lightblue' if val != 0 else 'white'
                rect = patches.Rectangle((j, 2-i), 1, 1, edgecolor='black', facecolor=color)
                ax.add_patch(rect)
                if val != 0:
                    ax.text(j+0.5, 2-i+0.5, str(val), ha='center', va='center', fontsize=16)
        title = f"Move: {move} | h: {h}" if move else f"Start | h: {h}"
        plt.title(title)
        plt.show()

# Random state generator
def generate_random_state():
    flat = list(range(9))
    while True:
        random.shuffle(flat)
        state = [flat[i:i+3] for i in range(0,9,3)]
        if is_solvable(flat):
            return state

def is_solvable(puzzle):
    inv = 0
    arr = [x for x in puzzle if x != 0]
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            if arr[i] > arr[j]:
                inv += 1
    return inv % 2 == 0

# Solve using BFS
def solve_bfs(start):
    visited = set()
    queue = deque()
    queue.append((start, [], manhattan(start)))
    visited.add(serialize(start))
    while queue:
        state, path, h = queue.popleft()
        if state == GOAL_8:
            return path
        for dir in ['up','down','left','right']:
            new_state = move(state, dir)
            if new_state and serialize(new_state) not in visited:
                visited.add(serialize(new_state))
                queue.append((new_state, path + [(dir, new_state)], manhattan(new_state)))
    return []

# Interactive Runner
class PuzzleStepper:
    def __init__(self):
        self.initial = generate_random_state()
        self.steps = solve_bfs(self.initial)
        self.index = 0
        self.total = len(self.steps)
        draw_puzzle(self.initial, "Start", manhattan(self.initial))

    def next_move(self, b):
        if self.index < self.total:
            move_name, state = self.steps[self.index]
            draw_puzzle(state, move_name, manhattan(state))
            self.index += 1
        else:
            with output:
                print("✅ Puzzle Solved!")

# Widgets
button = widgets.Button(description="Next Move")
output = widgets.Output()
ui = widgets.VBox([button, output])

stepper = PuzzleStepper()
button.on_click(stepper.next_move)

display(ui)